# Video Previewer with Pagination

This notebook allows you to **quickly preview some frames per video on an HPC** without downloading them locally. 

**Features:**
- Recursively looks for `.mp4` and `.avi` files within the chosen folder.
- Samples **5 evenly spaced frames** per video.
- Displays **parent_folder/video_file_name** as the figure title.
- Paginated preview: shows **10 videos at a time** with Next/Previous buttons.
- Interactive **folder chooser** to select the directory of videos.
- Lightweight and HPC-friendly (no GUI required, runs entirely in the notebook).

**Usage:**
1. Run the next cell to display the folder chooser.
2. Select a directory containing your videos.
3. The notebook will show 10 videos at a time, each with 5 frames.
4. Use **Next / Previous** buttons to navigate through all videos.

In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output
import ipywidgets as widgets

# --- Video sampling functions ---
def sample_frames(video_path, n_frames=5):
    """Sample evenly spaced frames from a video file."""
    cap = cv2.VideoCapture(str(video_path))
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    indices = np.linspace(0, frame_count - 1, n_frames, dtype=int)
    
    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frames.append((idx, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
    cap.release()
    return frames
    
# --- Show the videos
def show_frames(video_path, n_frames=5):
    """Display sampled frames from a video with frame indices and figure title."""
    frames = sample_frames(video_path, n_frames)
    fig, axes = plt.subplots(1, len(frames), figsize=(15, 3))
    for ax, (idx,img) in zip(axes, frames):
        ax.imshow(img)
        ax.set_title(f"Frame {idx}")
        ax.axis("off")
    fig.suptitle(f"{video_path.parent.name}/{video_path.name}")
    plt.show()

# --- Folder chooser ---
chooser = FileChooser()
chooser.title = "<b>Select Video Directory</b>"
chooser.show_only_dirs = True
display(chooser)

# --- Pagination buttons ---
videos = []
page_index = 0
videos_per_page = 10

btn_prev = widgets.Button(description="⬅ Previous")
btn_next = widgets.Button(description="Next ➡")
label_page = widgets.Label(value="Page 0 / 0")
output = widgets.Output()

controls = widgets.HBox([btn_prev, btn_next, label_page])
display(controls, output)

def update_page():
    """
    Update the current page of video previews.
    Displays videos_per_page videos starting from page_index.
    """
    with output:
        clear_output(wait=True)  # Only clear the video frame output
        start = page_index * videos_per_page
        end = start + videos_per_page
        for video_path in videos[start:end]:
            show_frames(video_path, n_frames=5)
        label_page.value = f"Page {page_index + 1} / {((len(videos)-1)//videos_per_page) + 1}"

def on_prev(_):
    """Go to the previous page if available."""
    global page_index
    if page_index > 0:
        page_index -= 1
        update_page()

def on_next(_):
    """Go to the next page if available."""
    global page_index
    if (page_index + 1) * videos_per_page < len(videos):
        page_index += 1
        update_page()

btn_prev.on_click(on_prev)
btn_next.on_click(on_next)

# --- Initialize video list when folder is chosen ---
def load_videos(_chooser=None):
    """
    Load videos from the selected folder using ipyfilechooser.
    Works with register_callback from ipyfilechooser.
    """
    global videos, page_index
    folder_path = chooser.selected_path  # use chooser.selected_path
    folder = Path(folder_path).expanduser()
    
    if folder.exists() and folder.is_dir():
        videos = sorted(list(folder.rglob("*.mp4")) + list(folder.rglob("*.avi")))
        page_index = 0
        if not videos:
            with output:
                clear_output()
                print(f"⚠️ No .mp4 or .avi files found in {folder}")
        else:
            update_page()
    else:
        with output:
            clear_output()
            print(f"❌ Directory not found: {folder}")

chooser.register_callback(load_videos)

FileChooser(path='/projects/kumar-lab/nguyetu/OFA_analysis/notebooks', filename='', title='<b>Select Video Dir…

Output()